In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from datetime import datetime
from IPython.display import clear_output

#Torch
from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS

# Additional torch-related imports
import torch
from torch import cat, no_grad, manual_seed
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch import nn
import torch.nn.functional as F



#Sklearn
from sklearn.preprocessing import MinMaxScaler

#Circuitos
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, transpile
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info.operators.predicates import is_unitary_matrix
from qiskit.quantum_info import SparsePauliOp # Necesito un obsevable en concreto...


from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZFeatureMap, ZZFeatureMap, QFT
from qiskit.circuit import ParameterVector
from qiskit.utils import algorithm_globals


# ML
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

manual_seed(444422)

In [19]:
n_qubits=3
qc = QuantumCircuit(n_qubits)

inputs = ParameterVector("inputs",n_qubits)
pesos = ParameterVector("pesos",2*n_qubits)
feature_map = QuantumCircuit(n_qubits)
feature_map.h(range(n_qubits))


for i in range(n_qubits):
    feature_map.ry(inputs[i],i)

for i in range(n_qubits-1):
    feature_map.cx(i,i+1)

for i in range(n_qubits):
    feature_map.ry(inputs[i],i)

ansatz = RealAmplitudes(n_qubits,reps=2)
#ansatz = QuantumCircuit(n_qubits)
#for i in range(n_qubits):
#    ansatz.ry(pesos[i],i)
#    
#for i in range(n_qubits-1):
#    ansatz.cz(i,i+1)
#
#for i in range(n_qubits,2*n_qubits):
#    ansatz.ry(pesos[i],i-n_qubits)



qc.compose(feature_map, inplace=True)
qc.compose(QFT(n_qubits, inverse = True), inplace=True)
qc.compose(ansatz, inplace=True)
qc.decompose().draw()

┌─────────┐┌──────────────────┐     ┌──────────────────┐»
q_0: ┤ U2(0,π) ├┤ R(inputs[0],π/2) ├──■──┤ R(inputs[0],π/2) ├»
     ├─────────┤├──────────────────┤┌─┴─┐└──────────────────┘»
q_1: ┤ U2(0,π) ├┤ R(inputs[1],π/2) ├┤ X ├─────────■──────────»
     ├─────────┤├──────────────────┤└───┘       ┌─┴─┐        »
q_2: ┤ U2(0,π) ├┤ R(inputs[2],π/2) ├────────────┤ X ├────────»
     └─────────┘└──────────────────┘            └───┘        »
«                            ┌───┐                         ┌──────────┐»
«q_0: ─────────────────────X─┤ H ├─■──────────────■────────┤ Ry(θ[0]) ├»
«     ┌──────────────────┐ │ └───┘ │P(-π/2) ┌───┐ │        └──────────┘»
«q_1: ┤ R(inputs[1],π/2) ├─┼───────■────────┤ H ├─┼──────────■─────────»
«     ├──────────────────┤ │                └───┘ │P(-π/4)   │P(-π/2)  »
«q_2: ┤ R(inputs[2],π/2) ├─X──────────────────────■──────────■─────────»
«     └──────────────────┘                                             »
«                                              ┌──────────┐                 »
«q_0: ──────────────────────────────────■──────┤ Ry(θ[3]) ├──────────■──────»
«     ┌──────────┐                    ┌─┴─┐    ├──────────┤        ┌─┴─┐    »
«q_1: ┤ Ry(θ[1]) ├──────────────■─────┤ X ├────┤ Ry(θ[4]) ├──■─────┤ X ├────»
«     └──┬───┬───┘┌──────────┐┌─┴─┐┌──┴───┴───┐└──────────┘┌─┴─┐┌──┴───┴───┐»
«q_2: ───┤ H ├────┤ Ry(θ[2]) ├┤ X ├┤ Ry(θ[5]) ├────────────┤ X ├┤ Ry(θ[8]) ├»
«        └───┘    └──────────┘└───┘└──────────┘            └───┘└──────────┘»
«     ┌──────────┐
«q_0: ┤ Ry(θ[6]) ├
«     ├──────────┤
«q_1: ┤ Ry(θ[7]) ├
«     └──────────┘
«q_2: ────────────
«

In [2]:
df=pd.read_csv('./datos/AirPassengers.csv')
df['Month']=pd.to_datetime(df['Month'], infer_datetime_format=True)
df=df.set_index(['Month'])

df_log=np.log(df)
data_shift=df_log-df_log.shift()
data_shift = data_shift.dropna()

In [3]:
ptrain=0.8
pasajeros = np.array(data_shift['#Passengers'])
pasajeros_train = pasajeros[:int(pasajeros.shape[0]*0.8)]
pasajeros_test = pasajeros[int(pasajeros.shape[0]*0.8):]

In [4]:
window = 12;
tam_train = pasajeros_train.shape[0] - window
tam_test = pasajeros_test.shape[0] - window

conjunto_train = []
objetivo_train = []

conjunto_test = []
objetivo_test = []

for i in range(0,tam_train):
    conjunto_train.append(pasajeros_train[0+i:window+i])
    objetivo_train.append(pasajeros_train[window+i])
    
for i in range(0,tam_test):
    conjunto_test.append(pasajeros_test[0+i:window+i])
    objetivo_test.append(pasajeros_test[window+i])

objetivo_train = np.array(objetivo_train)
conjunto_train = np.array(conjunto_train)

objetivo_test = np.array(objetivo_test)
conjunto_test = np.array(conjunto_test)

In [5]:
#scaler = MinMaxScaler((0.3, np.pi-0.3))
#conjunto_train = scaler.fit_transform(conjunto_train)
#objetivo_train = (-1.6/(np.min(objetivo_train) - np.max(objetivo_train)))*objetivo_train - 0.8 + np.min(objetivo_train)*(1.6/(np.min(objetivo_train) - np.max(objetivo_train)))

In [17]:
class ElecDataset(Dataset):
    def __init__(self,feature,target):
        self.feature = feature
        self.target = target
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self,idx):
        item = self.feature[idx]
        label = self.target[idx]
        
        return item,label

In [18]:
train = ElecDataset(conjunto_train.reshape(conjunto_train.shape[0],conjunto_train.shape[1],1),objetivo_train)
test = ElecDataset(conjunto_test.reshape(conjunto_test.shape[0],conjunto_test.shape[1],1),objetivo_test)
train_loader = torch.utils.data.DataLoader(train,batch_size=1,shuffle=False)
test_loader = torch.utils.data.DataLoader(test,batch_size=1,shuffle=False)

In [19]:
def circuitoRegresor_CodificacionAngular(n_qubits=1):
    inputs = ParameterVector("inputs",n_qubits)
    pesos = ParameterVector("pesos",9*n_qubits)
    feature_map = QuantumCircuit(n_qubits)
    
    for i in range(n_qubits):
            feature_map.ry(inputs[i],i)
    ansatz = QuantumCircuit(n_qubits)
    for j in range(3):
        for i in range(n_qubits):
            ansatz.rx(pesos[i + 3*j*n_qubits],i)

        for i in range(n_qubits):
            ansatz.ry(pesos[i + (3*j+1)*n_qubits],i)

        for i in range(n_qubits):
            ansatz.rz(pesos[i + (3*j+2)*n_qubits],i)

        for i in range(n_qubits-1):
            ansatz.cx(i,i+1)
    qc = QuantumCircuit(n_qubits)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)
    
    
    
    
    return feature_map.parameters, ansatz.parameters, qc, feature_map, ansatz

In [20]:
inputs, params, circuito, f1, f2 = circuitoRegresor_CodificacionAngular(2)

In [21]:
modelo_cuantico = EstimatorQNN(
        circuit=circuito, input_params=inputs, weight_params=params, input_gradients=True
    )

In [22]:


class Net(nn.Module):
    def __init__(self, qnn):
        super().__init__()
        self.fc1 = nn.Linear(12, 2)
        self.qnn = TorchConnector(modelo_cuantico)  # Apply torch connector, weights chosen
        # uniformly at random from interval [-1,1].
        self.fc2 = nn.Linear(1, 1)  # 1-dimensional output from QNN

    def forward(self, x):
        x = torch.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.qnn(x)  # apply QNN
        x = self.fc2(x)
        return x




In [23]:
redHibrida = Net(modelo_cuantico).double()
optimizer = torch.optim.Adam(redHibrida.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [24]:
# Numero de parametros
total_params = sum(
	param.numel() for param in redHibrida.parameters()
)
total_params

34

In [ ]:
redHibrida.train()
epochs = 200  # Set number of epochs
loss_list = []  # Store loss history

for epoch in range(epochs):
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad(set_to_none=True)  # Initialize gradient
        output = redHibrida(data)  # Forward pass
        #print("output ",output)
        #print("target ",target)
        loss = criterion(output, target)  # Calculate loss
        #print("loss ",loss)
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights
        total_loss.append(loss.item())  # Store loss
        #print('--------------------------------------------------------------')
    loss_list.append(sum(total_loss) / len(total_loss))
    print("Training {}/{}\tLoss: {:.20f}".format((epoch + 1), epochs ,loss_list[-1]))

Training 1/200	Loss: 0.04318880409276349558
Training 2/200	Loss: 0.01546802698724370623
Training 3/200	Loss: 0.01141656130288364740
Training 4/200	Loss: 0.01104332504070018248
Training 5/200	Loss: 0.01102457504146485238
Training 6/200	Loss: 0.01102705096086325012
Training 7/200	Loss: 0.01102950950685701974
Training 8/200	Loss: 0.01103141806736477465
Training 9/200	Loss: 0.01103295745359870149
Training 10/200	Loss: 0.01103423004099065609
Training 11/200	Loss: 0.01103529739698221393
Training 12/200	Loss: 0.01103620192123913601
Training 13/200	Loss: 0.01103697477790671448
Training 14/200	Loss: 0.01103763963218749866
Training 15/200	Loss: 0.01103821484463279705
Training 16/200	Loss: 0.01103871490467381658
Training 17/200	Loss: 0.01103915141383703460
Training 18/200	Loss: 0.01103953377966923090
Training 19/200	Loss: 0.01103986971620225799
Training 20/200	Loss: 0.01104016561160326565
Training 21/200	Loss: 0.01104042680270691558
Training 22/200	Loss: 0.01104065778305595227
Training 23/200	Los

In [ ]:
plt.plot(loss_list[:len(loss_list)-1],label='train_loss')
plt.title('MSE Loss')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

In [ ]:
entrada = conjunto_train.reshape(conjunto_train.shape[0],conjunto_train.shape[1],1)

In [ ]:
redHibrida.eval()
prediction = []
batch_size = 1
iterations =  int(entrada.shape[0]/batch_size)

for i in range(iterations):
    preds = redHibrida(torch.tensor(entrada[batch_size*i:batch_size*(i+1)]).double())
    prediction.append(preds[0].detach().numpy())
prediction = np.array(prediction)

In [ ]:
plt.plot(prediction, marker='^')
plt.plot(objetivo_train, marker='o')
plt.show()

In [ ]:
np.sum( (prediction - objetivo_train)**2 )/len(objetivo_train)

In [ ]:
entrada = conjunto_test.reshape(conjunto_test.shape[0],conjunto_test.shape[1],1)

In [ ]:
prediction = []
batch_size = 1
iterations =  int(entrada.shape[0]/batch_size)

for i in range(iterations):
    preds = redHibrida(torch.tensor(entrada[batch_size*i:batch_size*(i+1)]).double())
    prediction.append(preds[0].detach().numpy())
prediction = np.array(prediction)

In [ ]:
plt.plot(prediction, marker='^')
plt.plot(objetivo_test, marker='o')
plt.show()

In [ ]:
error = 0

for i in range(len(prediction)):
    error = error + (prediction[i] - objetivo_test[i])**2
error/len(prediction)